# News Summarizer
The idea is to build a News Articles Summarizer that will do the following:
- firstly get the latest news article with the help of the News API.
- then summarize the fetched news article using LLM under some word limit.

---

## Setup
You will need the API key for the News API which you can get from [here](https://newsdata.io/). You will have to create an account to proceed with generation of API key. Once you have the API key, you can define it in the `.env` file like this: `NEWSDATA_KEY="<your-api-key>"`

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_version = os.environ.get("OPENAI_API_VERSION")
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
from newsdataapi import NewsDataApiClient
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from time import sleep

## `NewsExtractor` class
Responsible for getting the news articles from the News API.

In [3]:
class NewsExtractor:
    """
    A class for extracting the latest news based on a search query.
    """

    def __init__(self, search_query: str = None):
        """
        Initializes the NewsExtractor instance.

        :param search_query: The text to be searched. Defaults to None.
        :type search_query: str, optional
        """
        self.search_query = search_query

    def get_news(self, input: str) -> str:
        """
        Gets a single latest news.

        :param input: Empty string, to be used by agent.
        :type query: str
        :return: A string containing the latest news.
        :rtype: str
        """
        import random

        api = NewsDataApiClient(apikey=os.environ.get("NEWSDATA_KEY"))

        if self.search_query:
            # If a search query is provided, retrieve the news based on the query, country (India), and language (English)
            response = api.news_api(q=self.search_query, country="in", language="en")
        else:
            # If no search query is provided, retrieve the general news for the country (India) and language (English)
            response = api.news_api(country="in", language="en")

        # Extract the content of each news item from the response and store it in the 'news' list
        news = [content["content"] for content in response["results"]]

        # Return a randomly selected news item from the 'news' list
        return random.choice(news)

## `news_summarizer` function

In [4]:
from typing import List
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI


def news_summarizer(
    limit: str = "175 characters",
    total_news: int = 5,
    sleep_seconds: int = 20,
    search_query: str = None,
) -> List[str]:
    """
    Function that retrieves the latest news and generates short summaries for the news using an AI model.

    :param limit: The maximum limit for each news summary. Defaults to "175 characters".
    :type limit: str, optional
    :param total_news: The number of news articles to summarize. Defaults to 5.
    :type total_news: int, optional
    :param sleep_seconds: The number of seconds to wait between each news retrieval and summarization. Defaults to 20.
    :type sleep_seconds: int, optional
    :param search_query: The text to be searched. Defaults to None.
    :type search_query: str, optional
    :return: A list of responses containing the generated news summaries.
    :rtype: list
    """
    # Initializing the AzureChatOpenAI model
    llm = AzureChatOpenAI(deployment_name="gpt4", temperature=0)

    # Initializing the LLMChain with the summarize_prompt
    summarize_prompt = PromptTemplate(
        input_variables=["news"], template="Summarize the following: {news}"
    )
    summarize_chain = LLMChain(llm=llm, prompt=summarize_prompt)

    # Initializing the NewsExtractor class with the provided search query
    news = NewsExtractor(search_query)

    # Initializing the tools
    tools = [
        Tool(
            name="get_news",
            func=news.get_news,
            description="gets a single news",
        ),
        Tool(
            name="summarize_news",
            func=summarize_chain.run,
            description="summarizes the news",
        ),
    ]

    # Initializing the agent with the tools and the AzureChatOpenAI model
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
    )

    # The prompt for the agent to generate the news summaries
    prompt = f"""
    Your task is to get a single news. Then generate a short summary for this news \
    in at most {limit}. Try to be as short as possible. Do not cross \
    the {limit} limit for summarized news.
    """

    responses = []

    for _ in range(total_news):
        response = agent.run(prompt)  # Running the agent to generate the news summary
        responses.append(response)
        sleep(sleep_seconds)  # to avoid rate limiting at OpenAI

    return responses  # Returning the list of generated news summaries

In [5]:
response = news_summarizer(limit="50 words", total_news=1, search_query="cricket")



> Entering new  chain...
I need to get a single news article and then summarize it in 50 words or less. I will first get the news article.
Observation: Invalid Format: Missing 'Action:' after 'Thought:'
Thought:I need to get a single news article and then summarize it in 50 words or less. I will first get the news article.
Action: get_news
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:I need to get a single news article and then summarize it in 50 words or less. I will first get the news article.
Action: get_news
Action Input: {}
Observation: Former India captain MS Dhoni 's decision to bat ahead of an in-form Yuvraj Singh in the 2011 ODI World Cup final against Sri Lanka turned out to be a game-changer. Knowing the Muttiah Muralitharan challenge that awaited the Indian team, Dhoni told the then-India coach Gary Kirsten that he plans to promote himself in the batting order. The rest, as they say, is history. More than a decade after the final, Muralitha

In [6]:
response

["Dhoni's decision to bat ahead of Yuvraj Singh in the 2011 ODI World Cup final was a game-changer. Familiar with Muralitharan's bowling from their time at Chennai Super Kings, Dhoni promoted himself, knowing Singh was uncomfortable playing against Muralitharan. This experience helped Dhoni tackle the spinner's challenge."]

In [7]:
len(response[0].split(" "))

47